# Day-ahead Bitcoin Price Forecasting using the Prophet model

This notebook builds a **full Prophet-based pipeline** for:

> **Day-ahead Bitcoin Price Forecasting (BTC-USD)**

We will:

- Clean and prepare a BTC-USD dataset (with some quirks in the raw CSV).
- Explore non-stationarity and volatility (returns & rolling volatility).
- Build a Prophet-ready dataset using **log of a Typical Price**.
- Train a **baseline additive Prophet model** and inspect trend/seasonality.
- Analyze residuals and link them to volatility (heteroskedasticity).
- Implement a **day-ahead forecasting protocol**:
  - Using **Prophet’s built-in `cross_validation`**.
  - Using a **manual rolling loop** for pedagogical purposes.
- Build an **improved multiplicative model** with tuned changepoints.
- Compare baseline vs improved in terms of:
  - MAE, RMSE, MAPE
  - **Directional accuracy** (up/down prediction of next-day move).
- Conclude by linking the results to Prophet theory:
  - \( y(t) = g(t) + s(t) + h(t) + \varepsilon_t \)


---

## 1. Import and Load the Bitcoin dataset

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from prophet import Prophet
from prophet.plot import add_changepoints_to_plot, plot_cross_validation_metric
from prophet.diagnostics import cross_validation, performance_metrics

from sklearn.metrics import mean_absolute_error, mean_squared_error

c:\Users\leoje\Desktop\Prophet_Project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


### Plotting style

In [2]:
plt.style.use("seaborn-v0_8")
plt.rcParams["figure.figsize"] = (12, 5)

### Display options

In [3]:
pd.set_option("display.max_columns", None)

### -----------------------------
## 1.1 Load raw BTC-USD.csv safely
### -----------------------------

In [ ]:
df_raw = pd.read_csv("BTC-USD.csv")

print("Original columns:", df_raw.columns.tolist())
display(df_raw.head(10))